<a href="https://colab.research.google.com/github/mariotito/CHAT_LLM_GEMINI/blob/main/IA_embbeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q -U google-generativeai

In [5]:
import numpy as np
import pandas as pd

import google.generativeai as genai

GOOGLE_API_KEY=""

genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
#Listando os modelos disponíveis
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


# Embed content

Call the embed_content method whit the *models/embedding-001* model to generate text embeddings.

In [10]:
# Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A próxima geração  de IA para desenvolvedores e Google Workspace"
    "\n"
    "Artigo Completo:\n"
    "\n"
    "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT"
                                 )

# Print just a part of the embedding to keep the output manageable
print(embeddings)

{'embedding': [0.05058498, -0.037521698, -0.018381886, 0.016740017, 0.06550638, -0.0069943094, -0.03152843, -0.016951015, 0.063357405, 0.061503988, 0.009029937, 0.016107513, -0.045594905, -0.022659285, 0.010208289, -0.025087189, 0.023489645, -0.01034037, -0.034423064, -0.0007657556, 0.0060251434, 0.0049782954, -0.02929734, -0.06921874, -0.017639026, 0.020024648, 0.01622423, -0.036543828, -0.033851583, 0.029656906, -0.04991703, 0.051595915, -0.03330382, 0.0120328665, -0.036838535, -0.0419492, -0.02812669, -0.043980617, -0.0015762714, -0.004804293, 0.0054791076, -0.08643493, -0.008798563, 0.025564311, -0.0057731024, -0.022547916, 0.047844518, 0.043846074, 0.015412018, -0.058614302, 0.028833145, 0.020551788, 0.0646853, -0.04256597, -0.0058407565, -0.006931777, 0.02380687, -0.03503929, 0.025705121, 0.00015910715, -0.001929766, 0.022371396, -0.018386982, 0.055253435, 0.020574134, -0.05734477, -0.041505445, -0.0011779857, 0.010356777, 0.04838751, 0.009132936, 0.012366308, 0.060436327, -0.062

In [9]:
print(len(result['embedding'])) # The embeddings have 768 dimensions

NameError: name 'result' is not defined

In [11]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Enchentes no Rio Grande do Sul",
    "Conteúdo": "Na região de Bento Gonçalves, mesmo em locais como a ERS-431, onde ainda havia uma ponte colapsada na enchente de setembro, novos danos ocorreram. Em Lajeado, a balsa foi levada pela correnteza, colidindo com a ponte do Rio Taquari. Na região, houve também danos em rodovias municipais, como na ponte da ERS-444, em Santa Tereza."}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incuindo navegação, entretenimento"}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada"}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [12]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Enchentes no Rio Grande do Sul,"Na região de Bento Gonçalves, mesmo em locais ..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [13]:
model = "models/embedding-001"

In [14]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT"
                             )["embedding"]


In [15]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Enchentes no Rio Grande do Sul,"Na região de Bento Gonçalves, mesmo em locais ...","[0.019781241, -0.00555663, -0.032384347, 0.004..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.016531516, -0.044613034, -0.016254714, -0.0..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.0021446403, -0.03835975, 0.0012374622, 0.0..."


In [17]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                             content=consulta,
                             task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [18]:
consulta = "quais municipios atingidos pela enchente?"

resposta = gerar_e_buscar_consulta(consulta, df, model)

print(resposta)

Na região de Bento Gonçalves, mesmo em locais como a ERS-431, onde ainda havia uma ponte colapsada na enchente de setembro, novos danos ocorreram. Em Lajeado, a balsa foi levada pela correnteza, colidindo com a ponte do Rio Taquari. Na região, houve também danos em rodovias municipais, como na ponte da ERS-444, em Santa Tereza.


In [24]:
generation_config = {
    "temperature": 0.6,
    "top_p": 0.95,
    "top_k": 35,
    "candidate_count": 1,
    }

In [25]:
prompt = f"reescreva esse texto de uma forma mais descontraída, sem adicionar informações que nao façam parte do texto: {resposta}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Eita, que bagunça na região de Bento Gonçalves! A ERS-431 tava com uma ponte caída desde setembro, e agora tem mais estrago. A balsa em Lajeado deu um rolê na correnteza e foi bater na ponte do Rio Taquari. E na ERS-444, em Santa Tereza, a ponte também não aguentou o tranco.
